In [23]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as kearas
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

In [2]:
interpreter = tf.lite.Interpreter("C:/Users/hjhhi/Desktop/Codes/lite-model_movenet_singlepose_lightning_tflite_float16_4.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()
output = interpreter.get_output_details()[0]  # Model has single output.
input = interpreter.get_input_details()[0]  # Model has single input.

# Make dataset for train

In [3]:
def make_image(size_1=32, size_2 = 32):
    path = "C:/Users/hjhhi/Downloads/image"
    y=[]
    data = []
    for i in os.listdir(path):
        for j in os.listdir(os.path.join(path, i)):
            temp = cv2.imread(os.path.join(path, i, j))
            temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
            temp = cv2.resize(temp, (size_1, size_2))
            data.append(temp)
            y.append(int(i))#Append int converting
    data = np.uint8(data)
    return data, y

In [4]:
X, y = make_image(192, 192)

In [5]:
X_ = []
for i in range(len(X)):
    input_data = X[i]
    interpreter.invoke()
    interpreter.set_tensor(input['index'], [input_data])
    X_.append(interpreter.get_tensor(output_details[0]['index']))
X_ = np.array(X_).reshape(len(X_), 51)

X: image resized by 192 for te_lite  
y: label for stretching pose  
X_: nparray returned by tflite by X
X_for_visualize: 480x640 image for visualize
X_train, X_test, y_train, y_test:   
모델 학습을 위하여 nparray를 쪼갬(이미지가 아니라는 점에 주의)

# Tuning model

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_, y)

In [7]:
model = RandomForestClassifier(n_estimators=50, max_depth=6)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9695652173913043

In [21]:
%%time
model.predict([X_[500]])

CPU times: total: 15.6 ms
Wall time: 5 ms


array([4])

# Save model by .pkl

In [24]:
joblib.dump(model, "./image_model.pkl")

['./image_model.pkl']

In [26]:
test_model = joblib.load("./image_model.pkl")